In [2]:
import os
from dfply import *
import pandas as pd
import datetime
np.set_printoptions(suppress=True) 
os.chdir(r"D:\03lecture\QT\codeBackup\data")
from math import *

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
def end_of_month(any_day):
    next_month = any_day.replace(day=28) + datetime.timedelta(days=4)
    return next_month - datetime.timedelta(days=next_month.day)

# formatting of msf
msf = pd.read_csv("msf.csv")
msf["date"] = pd.to_datetime(msf["date"], format='%Y%m%d')
msf["date"] = [end_of_month(date) for date in msf["date"]]
msf.columns = [i.lower() for i in msf.columns]
msf["month"] = [date.month for date in msf["date"]]
msf["year"] = msf.date.apply(lambda x:x.year)
# formatting of mse

In [5]:
mse = pd.read_csv("mse.csv")

In [6]:
mse.columns = [i.lower() for i in mse.columns]

In [7]:
mse["exdt"] = pd.to_datetime(mse["exdt"], format='%Y%m%d')
mse["dclrdt"] = pd.to_datetime(mse["dclrdt"], format='%Y%m%d')
mse["paydt"] = pd.to_datetime(mse["paydt"], format='%Y%m%d')
mse = mse >> mask(X.divamt == X.divamt) ## dropna subset == divamt

In [8]:
mse['date'] = mse['date'] = mse.exdt.agg(end_of_month)
# merge mse and msf

In [9]:
mse['freq'] = mse.distcd.apply(lambda x:str(x)[:3])
mse['freq'] = mse.freq.astype('int64')
mse['freq'].loc[mse['freq'] <= 123] = 123 ## 012全部作为quarterly data 对待

mseUse = pd.DataFrame(mse.groupby(['permno','date','distcd','freq'])['divamt'].sum()).reset_index()

In [10]:
data = pd.merge(msf,mseUse,how = 'left',on = ['permno','date'])

In [11]:
data.pop('bidlo')
data.pop('askhi')
data["mcap"] = data["prc"] * data["shrout"]
# fill na with distcd if the stock has distcd before so that we can identify stock that pay dividend
data = data >> arrange(X.permno,X.date)
data['freq'] = data.groupby('permno')['freq'].fillna(method = 'ffill')

In [12]:
data['distcd'] = data.groupby('permno')['distcd'].fillna(method = 'ffill')

In [13]:
divDataSource = data.drop_duplicates(subset = ['permno','date']) ## 如果同一个月即有123 又有非123的 保留，

In [14]:
## creat lag
divDataSource.sort_values(['permno','date'],inplace = True)
divDataSource['prc_lag1'] = divDataSource.groupby('permno')['prc'].shift(1)
divDataSource['mcap_lag1'] = divDataSource.groupby('permno')['mcap'].shift(1)
divDataSource['freq_lag1'] = divDataSource.groupby('permno')['freq'].shift(1)
divDataSource['turnover'] = divDataSource['vol']/divDataSource['shrout']
divDataSource['spread_2'] = divDataSource['ask'] -divDataSource['bid']

In [15]:
divDataSource['Div_Yield'] = divDataSource['divamt']/divDataSource['prc']

In [16]:
for i in range(12):
    div_lag = 'div_lag' + str(i + 1)
    divDataSource[div_lag] = divDataSource.groupby('permno')['divamt'].shift(i+1)

In [17]:
import feather

In [18]:
linkTable = feather.read_dataframe('monthly_gvkey_permno_link.feather')
linkTable['date'] = pd.to_datetime(linkTable['date'])

linkTable['N_permno']=linkTable.groupby('permno')['permno'].transform('count')
linkTable['year'] = linkTable.date.apply(lambda x:x.year)

linkTable.drop_duplicates(subset = ['gvkey','permno','year'],inplace = True)
linkTable['N_permno'] = linkTable.groupby(['year','gvkey'])['permno'].transform('count')
linkTable.query('N_permno >=2').sort_values(by = ['gvkey','year'])

BMdata = pd.read_csv('BMdata.csv')
BMdata['datadate'] = pd.to_datetime(BMdata.datadate,format = '%Y%m%d')
BMdata = BMdata[['gvkey','datadate','bkvlps']]

BMdata.rename(columns= {'datadate':'date'},inplace = True)
BMdata['year'] = BMdata.date.apply(lambda x:x.year)

BMdata_permno = BMdata.merge(linkTable,on = ['gvkey','year'],how= 'left',suffixes = ['_BM','_link'])
BMdata_permno.dropna(subset = ['bkvlps'],inplace= True)
BMdata_permno['permno'] = BMdata_permno.groupby('gvkey')['permno'].fillna(method = 'ffill')
BMdata_permno.drop_duplicates(subset = ['permno','date_BM','bkvlps'],inplace = True)
BMdata_permno.dropna(subset = ['permno'],inplace = True)
BMdata_permno['N_permno'] = BMdata_permno.groupby(['permno','date_BM'])['permno'].transform('count')
BMuse = pd.DataFrame(BMdata_permno.groupby(['permno','date_BM'])['bkvlps'].mean()).reset_index()
BMdata_permno.rename(columns = {'date_BM':'date'},inplace = True)

In [19]:
divDataSourceBM = pd.merge(divDataSource,BMdata_permno.drop(columns = ['year']),on = ['permno','date'],how = 'left')
divDataSourceBM = divDataSourceBM >> arrange(X.permno,X.date)
divDataSourceBM['bkvlps'] = divDataSourceBM.groupby('permno')['bkvlps'].fillna(method = 'ffill')
divDataSourceBM['bm'] = divDataSourceBM['bkvlps']/divDataSource['prc']

In [20]:
dataUse = divDataSourceBM.query('date <= "2011-12-31" &'
                          'shrcd in([10,11]) &'
                          'hexcd in([1,2,3]) &'
                          'prc_lag1 >= 5 &'
                          'ret not in(["B","C"])')

In [21]:
dataUse['ret'] = dataUse.ret.astype('float')

In [22]:
divData = dataUse.query('freq in(["120","121","123","124","125"])')

### 1 creat table1

#### 1.1  creat table1PanelA

In [23]:
table1PanelApart1 = divData[['mcap','turnover','spread_2','Div_Yield']].describe().T

In [24]:
table1PanelApart2 = divData.query('bm <500 & bm >0')[['bm']].describe().T

In [25]:
table1PanelA = pd.concat([table1PanelApart1,table1PanelApart2])

In [26]:
N_firms = pd.DataFrame({'count':len(set(divData['permno']))},index = ['N_firms'])

In [27]:
table1PanelA = pd.concat([table1PanelA,N_firms])

In [28]:
table1PanelA

,25%,50%,75%,count,max,mean,min,std
mcap,37168.812500,149425.000000,674204.625000,1312290.0,5.810989e+08,1.864001e+06,-1.300461e+08,1.077782e+07
turnover,0.129215,0.290577,0.652810,1316192.0,2.324402e+02,5.868285e-01,0.000000e+00,1.110118e+00
spread_2,0.080000,0.250000,0.500000,597116.0,1.724800e+02,4.261916e-01,-3.500000e+00,8.598707e-01
Div_Yield,0.004167,0.007692,0.012085,376759.0,2.727273e+00,8.342993e-03,-4.444444e-01,1.157757e-02
bm,0.415026,0.865341,1.966747,780765.0,4.875328e+02,2.566743e+00,6.909091e-05,9.163280e+00
N_firms,NaN,NaN,NaN,8326.0,NaN,NaN,NaN,NaN


#### 1.2 Creat table1PanelB

In [29]:
NoDivData = dataUse.query('freq not in(["120","121","123","124","125"])')

In [42]:
table1PanelBpart1 = NoDivData[['mcap','turnover','spread_2']].describe().T
table1PanelBpart2 = NoDivData.query('bm <1000&bm >0')[['bm']].describe().T

In [43]:
table1PanelBpart2

,count,mean,std,min,25%,50%,75%,max
bm,475330.0,1.756866,11.331695,0.000018,0.17903,0.433709,1.127327,950.742695


In [44]:
N_firms2 = pd.DataFrame({'count':len(set(NoDivData.permno))},index = ['N_firms'])

In [45]:
N_firms2

,count
N_firms,15917


In [46]:
table1PanelB= pd.concat([table1PanelBpart1,table1PanelBpart2,N_firms2])

In [47]:
table1PanelB

,25%,50%,75%,count,max,mean,min,std
mcap,43993.218750,140175.000000,464997.367500,693234.0,6.024329e+08,908446.377600,-1.776986e+07,6.180054e+06
turnover,0.366183,0.885089,1.915989,696685.0,3.041669e+02,1.545840,0.000000e+00,2.397334e+00
spread_2,0.050000,0.200000,0.375000,558218.0,6.500000e+02,0.369774,-3.312500e+00,4.313983e+00
bm,0.179030,0.433709,1.127327,475330.0,9.507427e+02,1.756866,1.845505e-05,1.133169e+01
N_firms,NaN,NaN,NaN,15917.0,NaN,NaN,NaN,NaN


### <font face = 'Times New Roman'> 1.3 Panel C column 1: Percent of firm-months with dividend in the last year

In [48]:
dataUse.columns

Index(['permno', 'date', 'shrcd', 'hexcd', 'prc', 'vol', 'ret', 'bid', 'ask',
       'shrout', 'spread', 'retx', 'month', 'year', 'distcd', 'freq', 'divamt',
       'mcap', 'prc_lag1', 'mcap_lag1', 'freq_lag1', 'turnover', 'spread_2',
       'Div_Yield', 'div_lag1', 'div_lag2', 'div_lag3', 'div_lag4', 'div_lag5',
       'div_lag6', 'div_lag7', 'div_lag8', 'div_lag9', 'div_lag10',
       'div_lag11', 'div_lag12', 'gvkey', 'bkvlps', 'date_link', 'N_permno',
       'bm'],
      dtype='object')

In [49]:
dataUse['divType'] = dataUse.distcd.apply(lambda x:str(x)[:3])

In [74]:
DivFreq = pd.DataFrame(dataUse.groupby(['year','divType'])['divType'].count())
DivFreq['percent'] = DivFreq.divType/DivFreq.groupby(level = 0).divType.transform(np.sum)
DivFreq.columns = ['count','percent']
DivFreq = DivFreq.reset_index()

In [76]:
DivFreq.head()

,year,divType,count,percent
0,1927,121,52,0.011670
1,1927,122,59,0.013241
2,1927,123,2580,0.578995
3,1927,124,75,0.016831
4,1927,127,168,0.037702


In [77]:
dataUse['divType'] = dataUse.groupby('permno')['divType'].fillna(method = 'ffill')

In [78]:
DivFreq= pd.DataFrame(DivFreq.groupby('divType')['percent'].mean())

In [79]:
div_freq = ['120','122','123','124','125','126','127']
Any_freq = DivFreq.loc[div_freq,].sum()
div_freq_index = {'monthly':'122','quarterly':'123','semiAnnualy':'124','Annualy':'125'}

In [80]:
DivFreq.head(10)

,percent
divType,
120,0.000066
121,0.162015
122,0.003524
123,0.562652
124,0.016181
125,0.010062
126,0.004743
127,0.028147
128,0.002464


In [81]:
table1PanelC_part1 = DivFreq.loc[['122','123','124','125','121'],]

In [82]:
table1PanelC_part1

,percent
divType,
122,0.003524
123,0.562652
124,0.016181
125,0.010062
121,0.162015


In [83]:
table1PanelC_part1.loc['Any_freq']= {'percent':Any_freq.tolist()[0]}

In [84]:
table1PanelC_part1

,percent
divType,
122,0.003524
123,0.562652
124,0.016181
125,0.010062
121,0.162015
Any_freq,0.625375


In [63]:
table1PanelC_part1

,percent
divType,
monthly,0.003524
quarterly,0.562652
semiAnnualy,0.016181
Annualy,0.010062
121,0.162015
Any_freq,0.625375


### <font face = 'Times New Roman'>  Panel C-Column 2: Percent of dividend observations

In [85]:
divData['divType'] = divData.distcd.apply(lambda x:str(x)[:3])

In [86]:
DivFreq2 = pd.DataFrame(divData.groupby('divType')['divType'].count())

In [87]:
DivFreq2.columns = ['count']

In [88]:
DivFreq2

,count
divType,
120,5
121,133836
122,3313
123,1118490
124,37516
125,24031


In [89]:
DivFreq2['percent'] = DivFreq2['count']/DivFreq2['count'].sum()

In [90]:
table1PanelC_part2 = DivFreq2[['percent']].loc[['122','123','124','125','121'],]

In [91]:
table1PanelC = pd.merge(table1PanelC_part1,table1PanelC_part2,left_index = True,right_index = True,how = 'outer')

In [92]:
table1PanelC.rename(index = {'121':'unknow_freq','122':'monthly','123':'quarterly','124':'semiAnnualy','125':'Annualy'},inplace = True)

In [93]:
table1PanelC = round(table1PanelC.apply(lambda x:x*100),2)

In [94]:
table1PanelC

,percent_x,percent_y
divType,,
unknow_freq,16.20,10.16
monthly,0.35,0.25
quarterly,56.27,84.91
semiAnnualy,1.62,2.85
Annualy,1.01,1.82
Any_freq,62.54,NaN


### 2  creat Table2

#### 2.1 creat Table2 PanelA

In [95]:
table2_mean = {}
table2_std = {}
table2_all_div_prob = {}
table2_quar_div_prob = {}

In [101]:
index_start = divData.columns.tolist().index('div_lag1')
index_end = divData.columns.tolist().index('div_lag12')

In [104]:
div_lag_list = divData.columns[index_start:index_end+1].tolist()

In [105]:
div_lag_list

['div_lag1',
 'div_lag2',
 'div_lag3',
 'div_lag4',
 'div_lag5',
 'div_lag6',
 'div_lag7',
 'div_lag8',
 'div_lag9',
 'div_lag10',
 'div_lag11',
 'div_lag12']

In [106]:
for i,div_lag in enumerate(div_lag_list):
    table2_mean[str(i + 1)] = divData.dropna(subset = [div_lag])['ret'].mean()*100
    table2_std[str(i + 1)] = divData.dropna(subset = [div_lag] )['ret'].std()*100
    table2_all_div_prob[str(i+1)] = len(divData.dropna(subset = [div_lag,'divamt']))/len(divData.dropna(subset = [div_lag]))
    divData_quar = divData.query('freq == "123"')
    table2_quar_div_prob[str(i + 1)] = len(divData_quar.dropna(subset = [div_lag,'divamt']))/len(divData_quar.dropna(subset = [div_lag]))

In [107]:
table2_panelA = pd.DataFrame(pd.Series(table2_mean),columns = ['Mean return'])
table2_panelA['Standard Deviation'] = table2_std.values()
table2_panelA['All dividends'] = table2_all_div_prob.values()
table2_panelA['quar dividends'] = table2_quar_div_prob.values()

In [108]:
table2_panelA#  ## quartly dividends 的return 有些偏高，因该是120，121，122都归类为123的原因，后面可以再调整下。

,Mean return,Standard Deviation,All dividends,quar dividends
1,1.047115,9.721843,0.010833,0.010986
2,1.191799,9.838917,0.062552,0.063514
3,1.372169,9.599297,0.858883,0.872877
4,1.007758,9.745201,0.064253,0.065057
5,1.197198,9.882160,0.068914,0.068576
6,1.389761,9.639035,0.855029,0.860230
7,1.031427,9.786326,0.069624,0.069264
8,1.171970,9.915469,0.067114,0.067989
9,1.367257,9.629136,0.840041,0.853276
10,1.007349,9.823710,0.067378,0.068255


#### 2.2 creat table2 PanelB

In [109]:
# assign signal
divData["signal"] = np.where((divData.div_lag3.notna() | divData.div_lag6.notna() |
                                 divData.div_lag9.notna() | divData.div_lag12.notna()), "L",
                                np.where(divData.div_lag1.notna() | divData.div_lag2.notna() |
                                         divData.div_lag4.notna() | divData.div_lag5.notna() |
                                         divData.div_lag7.notna() | divData.div_lag8.notna() |
                                         divData.div_lag10.notna() | divData.div_lag11.notna(), "S", ""))

portfolio1 = pd.DataFrame(divData.groupby(["date", "signal"])['ret'].mean())

In [110]:
probs = [0.01,0.05,0.25,0.5,0.75,0.95,0.99]

In [111]:
table2PanelB_part1 = portfolio1.groupby('signal')['ret'].describe(percentiles = probs).loc[['L','S']].drop(
    columns = ['count','max','min']).apply(lambda x:x*100)

In [112]:
table2PanelB_part1

,mean,std,1%,5%,25%,50%,75%,95%,99%
signal,,,,,,,,,
L,1.373574,5.928246,-16.403156,-7.681396,-1.438040,1.657452,4.384842,9.217188,16.602504
S,1.067091,6.066403,-17.101328,-8.112217,-1.658579,1.485880,4.068795,8.861298,15.205930


In [113]:
table2PanelB_part2 = pd.DataFrame(NoDivData.groupby('date')['ret'].mean()).describe(percentiles = probs).T.drop(columns = ['count','min','max']).apply(lambda x:x*100)

In [114]:
table2PanelB = pd.concat([table2PanelB_part1,table2PanelB_part2])

In [115]:
table2PanelB['index'] = ['div_predM','div_NoPredM','NoDiv']

In [116]:
table2PanelB.set_index('index',inplace = True)

In [142]:
def weighted_average(dataframe):
    return (dataframe.ret * dataframe.mcap_lag1).sum() / dataframe.mcap_lag1.sum()

In [143]:
divData.columns

Index(['permno', 'date', 'shrcd', 'hexcd', 'prc', 'vol', 'ret', 'bid', 'ask',
       'shrout', 'spread', 'retx', 'month', 'year', 'distcd', 'freq', 'divamt',
       'mcap', 'prc_lag1', 'mcap_lag1', 'freq_lag1', 'turnover', 'spread_2',
       'Div_Yield', 'div_lag1', 'div_lag2', 'div_lag3', 'div_lag4', 'div_lag5',
       'div_lag6', 'div_lag7', 'div_lag8', 'div_lag9', 'div_lag10',
       'div_lag11', 'div_lag12', 'gvkey', 'bkvlps', 'date_link', 'N_permno',
       'bm', 'divType', 'signal'],
      dtype='object')

In [144]:
port1 = divData.groupby(["date", "signal"]).apply(lambda x: pd.Series({"vwret": weighted_average(x),
                                                                                   "ewret": x.ret.mean()}))

In [145]:
port1

vwret     ewret
date       signal                    
1927-02-28         0.042561  0.045608
           S       0.031529  0.044375
1927-03-31         0.017802  0.004913
           S       0.006553 -0.020372
1927-04-30        -0.064381 -0.000682
           L       0.012585  0.003626
           S       0.007545 -0.003761
1927-05-31         0.044645  0.069592
           L       0.056182  0.038077
           S       0.056047  0.045857
1927-06-30        -0.011147 -0.021748
           L      -0.021385 -0.020012
           S      -0.025139 -0.018949
1927-07-31         0.069496  0.044435
           L       0.086880  0.048344
           S       0.068255  0.067024
1927-08-31         0.124403  0.124403
           L       0.037092  0.020785
           S       0.016657 -0.000190
1927-09-30         0.072937  0.064467
           L       0.051930  0.042971
           S       0.048872  0.036663
1927-10-31         0.010471  0.010471
           L      -0.037329 -0.014949
           S      -0.041401 -0.027990
1927-11-30         0.035916  0.028418
           L       0.067996  0.080155
           S       0.065554  0.081967
1927-12-31         0.028562  0.039792
           L       0.028494  0.037982
...                     ...       ...
2011-03-31        -0.004163  0.022399
           L       0.007746  0.017683
           S       0.003150  0.019495
2011-04-30         0.006127  0.012184
           L       0.030746  0.022847
           S       0.030605  0.017791
2011-05-31        -0.020577 -0.028322
           L      -0.013719 -0.017307
           S      -0.009504 -0.010216
2011-06-30        -0.036622 -0.031341
           L      -0.016014 -0.009078
           S      -0.013850 -0.009042
2011-07-31         0.029729 -0.040241
           L      -0.032388 -0.021749
           S      -0.025474 -0.026451
2011-08-31        -0.066855 -0.107950
           L      -0.052979 -0.060679
           S      -0.052733 -0.063335
2011-09-30        -0.070670 -0.135477
           L      -0.078560 -0.085305
           S      -0.067669 -0.089561
2011-10-31         0.122747  0.174697
           L       0.115840  0.133351
           S       0.108204  0.128094
2011-11-30        -0.041871 -0.017587
           L       0.003779  0.005891
           S       0.003982  0.002318
2011-12-31         0.028137  0.001142
           L       0.015590  0.005557
           S       0.009426  0.008063

[3055 rows x 2 columns]

In [146]:
port1.reset_index(inplace=True)
# calculate the total return of equal weight and value weight if we long and short each month
port1 = port1.query('signal in(["L","S"])').groupby("date").apply(
                          lambda x: pd.Series({"ewret": sum(np.where(x.signal == "L", x.ewret, x.ewret*-1)),
                                               "vwret": sum(np.where(x.signal == "L", x.vwret, x.vwret*-1))}))

In [195]:
long1_short2 = port1[['ewret']].describe(percentiles = probs).T.drop(columns = ['count','max','min']).apply(lambda x:x*100)

In [ ]:
long1_short2.insert(loc = 2)

In [181]:
port_long = pd.DataFrame(divData.query('signal == "L"').groupby('date')['ret'].mean())

In [184]:
port_short = pd.DataFrame(NoDivData.groupby('date')['ret'].mean())

In [191]:
portfolio2 = port_long.join(port_short,lsuffix = '_long',rsuffix = '_short') >> mutate(monthly_ret = X.ret_long - X.ret_short)

In [194]:
long1_short3 = portfolio2[['monthly_ret']].describe(percentiles = probs).T.drop(columns = ['count','max','min']).apply(lambda x:x*100)

In [208]:
table2PanelB

,mean,std,1%,5%,25%,50%,75%,95%,99%
index,,,,,,,,,
div_predM,1.373574,5.928246,-16.403156,-7.681396,-1.438040,1.657452,4.384842,9.217188,16.602504
div_NoPredM,1.067091,6.066403,-17.101328,-8.112217,-1.658579,1.485880,4.068795,8.861298,15.205930
NoDiv,0.980859,8.370443,-23.399086,-11.848510,-3.330714,1.536198,5.426408,13.156111,21.093995


In [204]:
long1_short2['index'] = 'long1_short2'

In [210]:
long1_short2.set_index('index',inplace = True)

In [212]:
long1_short2

,mean,std,1%,5%,25%,50%,75%,95%,99%
index,,,,,,,,,
long1_short2,0.303787,0.822482,-1.767952,-0.896306,-0.135959,0.282719,0.739372,1.527203,2.605708


In [213]:
long1_short3['index'] = 'long1_short3'

In [214]:
long1_short3.set_index('index',inplace = True)

In [215]:
long1_short3

,mean,std,1%,5%,25%,50%,75%,95%,99%
index,,,,,,,,,
long1_short3,0.395042,3.603616,-9.007057,-5.111056,-1.363388,0.341944,2.189092,5.778831,9.465052


In [217]:
table2PanelB = pd.concat([table2PanelB,long1_short2,long1_short3])

In [219]:
table2PanelB.insert(loc= 2,column = 'sharp_ratio',value = (table2PanelB['mean']-0.297)/table2PanelB['std'])

In [220]:
table2PanelB

,mean,std,sharp_ratio,1%,5%,25%,50%,75%,95%,99%
index,,,,,,,,,,
div_predM,1.373574,5.928246,0.181601,-16.403156,-7.681396,-1.438040,1.657452,4.384842,9.217188,16.602504
div_NoPredM,1.067091,6.066403,0.126944,-17.101328,-8.112217,-1.658579,1.485880,4.068795,8.861298,15.205930
NoDiv,0.980859,8.370443,0.081699,-23.399086,-11.848510,-3.330714,1.536198,5.426408,13.156111,21.093995
long1_short2,0.303787,0.822482,0.008252,-1.767952,-0.896306,-0.135959,0.282719,0.739372,1.527203,2.605708
long1_short3,0.395042,3.603616,0.027207,-9.007057,-5.111056,-1.363388,0.341944,2.189092,5.778831,9.465052


In [222]:
portfolio2['cumulative_ret'] = np.cumprod(1+portfolio2['monthly_ret'])

In [227]:
port1['cumulative_ret_ewret'] = np.cumprod(1+port1['ewret'])

In [228]:
port1.tail()

,ewret,vwret,cumulative_ret_ewret
date,,,
2011-08-31,0.002656,-0.000245,21.032634
2011-09-30,0.004255,-0.010892,21.122131
2011-10-31,0.005257,0.007635,21.233173
2011-11-30,0.003573,-0.000203,21.309041
2011-12-31,-0.002505,0.006164,21.255659


In [224]:
portfolio2.tail()

,ret_long,ret_short,monthly_ret,cumulative_ret
date,,,,
2011-08-31,-0.060679,-0.106342,0.045663,27.118517
2011-09-30,-0.085305,-0.112493,0.027187,27.855799
2011-10-31,0.133351,0.139939,-0.006588,27.672290
2011-11-30,0.005891,-0.016764,0.022655,28.299201
2011-12-31,0.005557,0.004643,0.000915,28.325092
